# EDA

Importing main libraries

In [ ]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import tensorflow as tf

Here we just read our dataframe and delete the first column which is useless in training phase

In [ ]:
dataframe = pd.read_csv('/kaggle/input/iba-ml1-final-project/train.csv')
dataframe.drop(["Id"], axis = 1, inplace = True)
dataframe

When training the model you can see that the overfitting happening easily, it is because we have little data for the NLP task, and we know that one of the best ways of dealing against overfitting is adding the new data to our dataframe. That is why I have added new data, basically united two dataframe (our competition data, and another data which available in Kaggle). After training the model with this united dataframe, the model gives very very good results.

Basically in this cell we read that dataframe, change appropriate columns and append it to our main dataframe.However due to new requirement,  this cell was commented, and it will not used in model fitting

In [ ]:
#dataframe_extra = pd.read_csv('/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv')
#dataframe_extra.drop(['Unnamed: 0', 'Clothing ID'], axis = 1, inplace = True)
#dataframe_extra = dataframe_extra.rename(columns={'Title': 'Review_Title',
#                                                  'Review Text': 'Review',
#                                                  'Recommended IND':'Recommended',
#                                                  'Positive Feedback Count':'Pos_Feedback_Cnt',
#                                                  'Division Name':'Division',
#                                                  'Department Name':'Department',
#                                                  'Class Name':'Product_Category'})
#dataframe = dataframe.append(dataframe_extra).reset_index().drop("index",axis=1)

Here we just one hot encode the Rating target value with using get_dummies method in pandas.

In [ ]:
one_hot_Rating = pd.get_dummies(dataframe.Rating,prefix="Rating")
dataframe = dataframe.drop('Rating',axis = 1)
dataframe = dataframe.join(one_hot_Rating)
dataframe

Let's find and drop the records which do not have both Review and Review title, because if both of them are null at the same time, there is no way that we can know what is our target values

In [ ]:
dataframe.drop(dataframe[dataframe.Review_Title.isnull() & dataframe.Review.isnull()].index, inplace = True)
dataframe = dataframe.reset_index().drop('index',axis =1)
dataframe

Here we write our new function which just converts the pos_tag which nltk found for the word to wordnet pos_tag. We will need it for lemmatization.

In [ ]:
from nltk.corpus import wordnet

def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0]
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    element = tag_dict.get(tag)
    if(element == "J" or element=="N" or element=="V" or element=="R"):
        return element
    return 'n' #it is just like for not getting the error, we pass 'n' if the pos tag couldn't found or is not in the dictionary.

In this cell, First we filling missin values with None, and then we apply the classical NLP techniques to the Review and Review_Title columns. First we make all of them lower string, later we remove the stop words, and then lemmatize the review or review_title.

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


dataframe['Review_Title'] = dataframe['Review_Title'].fillna('None')
dataframe['Review'] = dataframe['Review'].fillna('None')
pd.set_option('mode.chained_assignment', None)
dataframe.Review = dataframe.Review.str.lower()
dataframe.Review_Title = dataframe.Review_Title.str.lower()
for i in range(0,len(dataframe)):
    review = re.sub('[^a-zA-z]', ' ', dataframe['Review'][i])
    review = review.split()
    wl = WordNetLemmatizer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')    
    review = [wl.lemmatize(word, pos = get_pos(word)) for word in review if word not in set(all_stopwords)]
    review = ' '.join(review)
    dataframe['Review'][i] = review
    
    review_title = re.sub('[^a-zA-z]', ' ', dataframe['Review_Title'][i])
    review_title = review_title.split()
    review_title = [wl.lemmatize(word, pos = get_pos(word)) for word in review_title if word not in set(all_stopwords)]
    review_title = ' '.join(review_title)
    dataframe['Review_Title'][i] = review_title

In [ ]:
dataframe[['Review_Title','Review']]

Let's split our data to train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_dataframe = dataframe.iloc[:,:-6]
y_dataframe = dataframe.iloc[:,-6:]

X_train, X_test, y_train, y_test = train_test_split(X_dataframe,y_dataframe, stratify = y_dataframe.Recommended)

We will use functional API for that problem, that is why we wil have 3 different input, and two output. For inputs one for the Review_Title, another for Review, and the last one for other columns. And for outputs one for Recommended and the other for Review

In [ ]:
X_train_review_title_input = X_train[['Review_Title']]
X_train_review_input = X_train[['Review']]
X_train_other_input = X_train[['Age','Pos_Feedback_Cnt','Division','Department','Product_Category']]

X_test_review_title_input = X_test[['Review_Title']]
X_test_review_input = X_test[['Review']]
X_test_other_input = X_test[['Age','Pos_Feedback_Cnt','Division','Department','Product_Category']]

y_train_rating_output = y_train[['Rating_1','Rating_2','Rating_3','Rating_4','Rating_5']]
y_train_recommended_output = y_train[['Recommended']]


y_test_rating_output = y_test[['Rating_1','Rating_2','Rating_3','Rating_4','Rating_5']]
y_test_recommended_output = y_test[['Recommended']]

Now let's do some preprocessing steps for the other columns train input. For it, we will create a pipeline and do some appropriate steps (scaling, imputing) for numerical and categorical columns.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer


prep_other_pipe = Pipeline(steps=[
    ('preprocessing',ColumnTransformer(transformers=[
        ('numeric', Pipeline(steps=[
           ('impute', SimpleImputer(strategy='mean')),
            ('scaler', RobustScaler())
        ]), ['Age']),
        ('categorical', Pipeline(steps=[
            ('impute', SimpleImputer(strategy='most_frequent')),
            ('one_hot_encoding', OneHotEncoder(handle_unknown = 'ignore', sparse = False))
        ]), ['Pos_Feedback_Cnt','Division','Department','Product_Category'])
    ]))
])
X_train_other_input = prep_other_pipe.fit_transform(X_train_other_input)
X_test_other_input = prep_other_pipe.transform(X_test_other_input)

In [ ]:
X_train_other_input

Now let's Vectorize our Review and Review_Title columns. I will use 500  Vocabulary size for Review Title and 10000 Vocabulary size for Review

In [ ]:
Vocab_Size_Review_Title = 5000
encoder_RTitle = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=Vocab_Size_Review_Title)
encoder_RTitle.adapt(np.array(X_train_review_title_input))
#encoder_RTitle.get_vocabulary()

In [ ]:
Vocab_Size_Review = 10000
encoder_Review = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=Vocab_Size_Review)
encoder_Review.adapt(np.array(X_train_review_input))
#encoder_Review.get_vocabulary()

# Model

Now let's create our model. As I said, we will use function api for it.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

input_review_title = keras.Input(shape=(None,), name="review_title")
input_review = keras.Input(shape=(None,), name="review")
input_other_columns = keras.Input(shape=(X_train_other_input.shape[1],), name="other_columns")

embedding_review_title = layers.Embedding(len(encoder_RTitle.get_vocabulary()),500)(input_review_title)
embedding_review = layers.Embedding(len(encoder_Review.get_vocabulary()),500)(input_review)

lstm_review_title = layers.LSTM(16,name = 'lstm_review_title')(embedding_review_title)
lstm_review = layers.LSTM(32,name = 'lstm_review')(embedding_review)
dense_other_columns = layers.Dense(2, name='dense_other_columns')(input_other_columns)

concat_layer = layers.concatenate([lstm_review_title,lstm_review,dense_other_columns])

rating_pred = layers.Dense(5,activation ='softmax',name="rating")(concat_layer)
recommended_pred = layers.Dense(1,activation ='sigmoid',name="recommended")(concat_layer)

model = keras.Model(
    inputs=[input_review_title, input_review,input_other_columns],
    outputs=[rating_pred, recommended_pred],
)

Let's visualize our model

In [ ]:
keras.utils.plot_model(model,show_shapes=True)

We will use adam optimizer, and as a loss functions we will use categorical_crossentropy for rating and binary_crossentropy for recommended. And as a metric we will use accuracy score for both of them

In [ ]:
model.compile(optimizer='adam', loss = {"rating": "categorical_crossentropy",
                                        "recommended": "binary_crossentropy"},
                                metrics={'rating': 'accuracy', 
                                         'recommended': 'accuracy'})

We will have two callback, one for loss and other for validation rating accuracy with patience 3. And for fitting the mmodel we will have 25 epoch with batch size 32

In [ ]:
callback_loss = tf.keras.callbacks.EarlyStopping(monitor='loss',patience = 5)
callback_accuracy = tf.keras.callbacks.EarlyStopping(monitor='val_rating_accuracy', patience = 3)
history = model.fit({"review_title": encoder_RTitle(X_train_review_title_input),
                     "review": encoder_Review(X_train_review_input),
                     "other_columns": X_train_other_input}, 
                    {"rating": y_train_rating_output,
                     "recommended":y_train_recommended_output}, 
                    epochs=30,batch_size=128, verbose=0, 
                    validation_data = ({"review_title": encoder_RTitle(X_test_review_title_input),
                                        "review": encoder_Review(X_test_review_input),
                                        "other_columns": X_test_other_input},
                                         {"rating": y_test_rating_output,
                                           "recommended":y_test_recommended_output}),
                   callbacks=[callback_loss, callback_accuracy])

# Interpretation

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())

fig, axs = plt.subplots(2, 2, figsize = (20,10))
axs[0, 0].plot(history.history['rating_loss'])
axs[0, 0].set_title('rating_loss')
axs[0, 1].plot(history.history['recommended_loss'], 'tab:orange')
axs[0, 1].set_title('recommended_loss]')
axs[1, 0].plot(history.history['val_rating_loss'], 'tab:green')
axs[1, 0].set_title('val_rating_loss')
axs[1, 1].plot(history.history['val_recommended_loss'], 'tab:red')
axs[1, 1].set_title('val_recommended_loss')

## Applying to the test set

In these following 5 columns, we will do the same EDA processes for our test set

In [ ]:
test_dataframe = pd.read_csv('/kaggle/input/iba-ml1-final-project/test.csv')
test_dataframe_Id = test_dataframe["Id"]
test_dataframe.drop(["Id"], axis = 1, inplace = True)
test_dataframe

Applying the same NLP techniques

In [ ]:
test_dataframe['Review_Title'] = test_dataframe['Review_Title'].fillna('None')
test_dataframe['Review'] = test_dataframe['Review'].fillna('None')
#Now
test_dataframe.Review = test_dataframe.Review.str.lower()
test_dataframe.Review_Title = test_dataframe.Review_Title.str.lower()
for i in range(0,len(test_dataframe)):
    review = re.sub('[^a-zA-z]', ' ', test_dataframe['Review'][i])
    review = review.split()
    wl = WordNetLemmatizer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')    
    review = [wl.lemmatize(word, pos = get_pos(word)) for word in review if word not in set(all_stopwords)]
    review = ' '.join(review)
    test_dataframe['Review'][i] = review
    
    review_title = re.sub('[^a-zA-z]', ' ', test_dataframe['Review_Title'][i])
    review_title = review_title.split()
    review_title = [wl.lemmatize(word, pos = get_pos(word)) for word in review_title if word not in set(all_stopwords)]
    review_title = ' '.join(review_title)
    test_dataframe['Review_Title'][i] = review_title

Seperate columns for input

In [ ]:
test_df_review_title_input = test_dataframe[['Review_Title']]
test_df_review_input = test_dataframe[['Review']]
test_df_other_input = test_dataframe[['Age','Pos_Feedback_Cnt','Division','Department','Product_Category']]

Transform them (imputing, scaling) with using the same object of main dataframe

In [ ]:
test_df_other_input = prep_other_pipe.transform(test_df_other_input)

Now let's predict our model

In [ ]:
test_pred = model.predict({"review_title": encoder_RTitle(test_df_review_title_input),
                           "review": encoder_Review(test_df_review_input),
                           "other_columns": test_df_other_input})

Change the probabilities to appropriate numbers.

In [ ]:
rating_answer = np.argmax(np.array(test_pred[0]),axis=1)+1
recom_answer = (test_pred[1] > 0.5).astype(int).flatten()

Let's put them to the pandas dataframe with proper columns

In [ ]:
df_answer = pd.DataFrame({"Id":test_dataframe_Id,
                          "Rating":rating_answer,
                          "Recommended": recom_answer})
df_answer